In [1]:
import pandas as pd

In [2]:
arquivoValoresZerados = 'C:\\Temp\\RT\\VALORES_ZERADOS.xlsx'
arquivoDatabricks = 'C:\\Temp\\RT\\CARGA_DATABRICKS.csv'

In [3]:
dffat_indicadores_frete_ZLIQ  = pd.read_csv(arquivoDatabricks, sep=',', encoding='latin-1'
                                            ,usecols=[8,9,10,39,40]
                                            ,names=['NumDocContabil'
                                              , 'NumDocCompra'
                                              , 'CodDocVenda'
                                              , 'VlrLiqItemOrdem'
                                              , 'VlrPesoLiqItem']
                                                ,dtype={'NumDocContabil':'str'
                                              , 'NumDocCompra':'str'
                                              , 'CodDocVenda':'str'
                                              , 'VlrLiqItemOrdem':'float64'
                                              , 'VlrPesoLiqItem':'float64'}
                                            ,skiprows=1
                                            ,header=None
                                            ,low_memory=False)


dfValoresZerados  = pd.read_excel(arquivoValoresZerados
                                 , usecols=[0,1,2,5,6,7,8]
                                  ,names=['CodDocVenda'
                                          , 'NumDocCompra'
                                          , 'NumDocContabil'
                                          , 'VlrLiqItemOrdem'
                                          , 'VlrPesoLiqItem'
                                          , 'VlrPesoLiqItem_Bruno'
                                          , 'VlrLiqItemOrdem_Bruno']
                                  ,dtype={'CodDocVenda':'str'
                                          , 'NumDocCompra':'str'
                                          , 'NumDocContabil':'str'
                                          , 'VlrLiqItemOrdem':'float64'
                                          , 'VlrPesoLiqItem':'float64'
                                          , 'VlrPesoLiqItem_Bruno':'float64'
                                          , 'VlrLiqItemOrdem_Bruno':'float64'}
                                 )

In [4]:
dfValoresZerados['Chave'] = dfValoresZerados['CodDocVenda'] + '_' + \
                                            dfValoresZerados['NumDocCompra'] + '_' + \
                                            dfValoresZerados['NumDocContabil']

dffat_indicadores_frete_ZLIQ['Chave'] = dffat_indicadores_frete_ZLIQ['CodDocVenda'] + '_' + \
                                            dffat_indicadores_frete_ZLIQ['NumDocCompra'] + '_' + \
                                            dffat_indicadores_frete_ZLIQ['NumDocContabil']

dffat_indicadores_frete_ZLIQ_agg = dffat_indicadores_frete_ZLIQ.groupby(['CodDocVenda'
                                      ,'NumDocCompra'
                                      ,'NumDocContabil'
                                      ,'Chave'],as_index=False).agg(VlrLiqItemOrdem_y=('VlrLiqItemOrdem','sum')
                                                     ,VlrPesoLiqItem_y=('VlrPesoLiqItem','sum'))[['CodDocVenda'
                                                                                              , 'NumDocCompra'
                                                                                              , 'NumDocContabil'
                                                                                              , 'Chave'
                                                                                              , 'VlrLiqItemOrdem_y'
                                                                                              , 'VlrPesoLiqItem_y']]


dffat_indicadores_frete_ZLIQ_agg['VlrLiqItemOrdem_y'] = dffat_indicadores_frete_ZLIQ_agg['VlrLiqItemOrdem_y'].astype('float',errors='ignore')
dffat_indicadores_frete_ZLIQ_agg['VlrLiqItemOrdem_y'] = dffat_indicadores_frete_ZLIQ_agg['VlrLiqItemOrdem_y'].astype('float',errors='ignore')

In [5]:
resultLiqItem = pd.merge(left=dfValoresZerados
                              , right=dffat_indicadores_frete_ZLIQ_agg
                              , left_on='Chave'
                              , right_on='Chave' 
                              , how='left'
                                    )[['CodDocVenda_x'
                                      , 'NumDocCompra_x'
                                      , 'NumDocContabil_x'
                                      , 'Chave'
                                      , 'VlrLiqItemOrdem_y'
                                      , 'VlrLiqItemOrdem_Bruno'                                       
                                      ]]

resultLiqItem.fillna(0,inplace=True)


resultPesoLiq = pd.merge(left=dfValoresZerados
                              , right=dffat_indicadores_frete_ZLIQ_agg
                              , left_on='Chave'
                              , right_on='Chave' 
                              , how='left'
                                    )[['CodDocVenda_x'
                                      , 'NumDocCompra_x'
                                      , 'NumDocContabil_x'
                                      , 'Chave'
                                      , 'VlrPesoLiqItem_y'
                                      , 'VlrPesoLiqItem_Bruno'                                       
                                      ]]

resultPesoLiq.fillna(0,inplace=True)

In [6]:
resultPesoLiq.loc[resultPesoLiq['VlrPesoLiqItem_y'] == resultPesoLiq['VlrPesoLiqItem_Bruno'], 'Comparacao_VlrPesoLiqItem'] = 'Igual'
resultPesoLiq.loc[resultPesoLiq['VlrPesoLiqItem_y'] != resultPesoLiq['VlrPesoLiqItem_Bruno'], 'Comparacao_VlrPesoLiqItem'] = 'Diferente'

resultLiqItem.loc[resultLiqItem['VlrLiqItemOrdem_y'] == resultLiqItem['VlrLiqItemOrdem_Bruno'], 'Comparacao_VlrLiqItemOrdem'] = 'Igual'
resultLiqItem.loc[resultLiqItem['VlrLiqItemOrdem_y'] != resultLiqItem['VlrLiqItemOrdem_Bruno'], 'Comparacao_VlrLiqItemOrdem'] = 'Diferente'

In [13]:
#resultPesoLiq[resultPesoLiq['Comparacao_VlrPesoLiqItem']=='Diferente'].head(5)

resultPesoLiq[resultPesoLiq['CodDocVenda_x']=='4000240586'].head(5)

,CodDocVenda_x,NumDocCompra_x,NumDocContabil_x,Chave,VlrPesoLiqItem_y,VlrPesoLiqItem_Bruno,Comparacao_VlrPesoLiqItem


In [8]:
resultLiqItem[resultLiqItem['Comparacao_VlrLiqItemOrdem']=='Diferente'].to_csv('Comparacao_VlrLiqItemOrdem.csv')
#.head(5)

In [12]:
resultLiqItem[resultLiqItem['CodDocVenda_x']=='4000240586'].head(5)

,CodDocVenda_x,NumDocCompra_x,NumDocContabil_x,Chave,VlrLiqItemOrdem_y,VlrLiqItemOrdem_Bruno,Comparacao_VlrLiqItemOrdem
